In [ ]:
# 拖船分派演算法

## 貪婪 (greedy algorithm)
import itertools as it 
import pandas as pd
import math

from datetime import datetime  
from datetime import timedelta

from enum import Enum, IntEnum

## Self-defined function
from WorkTimePrediction import *

class Event():
    def __init__():
        pass
    def event_handler():
        pass

class Event(Enum):
    """All kinds of events which trigger planning algorithm
    
    Attributes:
        ROUTINE:            dispatch every one hour
        SHIP_DELAY:         delay caused by the ship
        TUG_DELAY:          delay caused by the tug
        UNIDENTIFIED_DELAY: delay occurs under unidentified situation
        MISMATCH:           the previous assignment is refused by the pilot
        BEFOREHAND:         any of the tugs finish task in advance
        CANCELED:           the task is canceled
        ADD_TUG:            need additional tugs on the half way
    """
    
    ROUTINE            = 'routine' 
    CANCELED           = 'canceled'
    TUG_DELAY          = 'tug_delay'
    SHIP_DELAY         = 'ship_delay'
    UNIDENTIFIED_DELAY = 'unidentified_delay'
    
    MISMATCH           = 'mismatch'
    BEFOREHAND         = 'beforehand'
    ADD_TUG            = 'add_tug'
    
    
class TUG_STATE(Enum):
    """States of tugs which are considered when planning
    
    Attributes:
        FREE:        a tug which is ready to work immediately
        BUSY:        a tug which is busy working on a specific tesk
        UNAVAILABLE: a tug which is either broken or out of order
    """
    FREE        = 0
    BUSY        = 1
    UNAVAILABLE = 2

class CHARGE_TYPE(IntEnum):
    """Types of tugs which are used in charging and planning.
        Values of attributes correpond to weights according to the dispatching rule
    """
    
    TYPE_117 = 117
    TYPE_118 = 118
    TYPE_119 = 119
    TYPE_120 = 120
    TYPE_0   = 130
    
class TASK_STATE(IntEnum):
    """States of task describing 
        if tugs are assigned (UNASSIGNED or ASSIGNED)
        if the task is finished (UNPROCESSED, PROCESSING, or PROCESSED)
    """
    
    UNPROCESSED_UNASSIGNED = 0
    UNPROCESSED_ASSIGNED   = 1
    PROCESSING_ASSIGNED    = 2
    PROCESSED              = 3
    
class SIDE(Enum):
    LEFT     = 'L' 
    RIGHT    = 'R'
    OPTIONAL = 'O'

class SHIP_STATE(Enum):
    IN        = 'I'
    OUT       = 'O'
    TRANSFER  = 'T'
    
class TUG():
    """Main entities to be planned
    
    Attributes:
        _id (int):           ID of the tug
        _pos ((float, float)):   current position of the tug, a pair of latitude and lontitude
        _type (CHARGE_TYPE): charging type
        _hp (int):           horsepower of the tug
        _velo (int):         velocity, default 10kt
        _state (TUG_STATE):  current state of the tug
        _next_available_time: if tug state is busy,we will recorded the time which it will finish the task
    """
    
    def __init__(self, tug_id, cur_pos, charge_type, 
                hp, next_available_time ,state=TUG_STATE.FREE, velo=10):
        self._tug_id = tug_id
        self._pos    = cur_pos
        self._type   = charge_type
        self._hp     = hp
        self._state  = state
        self._velo   = velo
        self._next_avaliable_time = next_available_time
        
def return_type(elem):
            return (elem._type)
class TUG_LIST(object):
    
    _tug_list = []
    
    def __init__(self, tug_list):
        self._tug_list = tug_list
        
    def __lt__(self, other):
        return(not(self > other) and not(self == other))

    def __eq__(self, other):
        self = sorted(self._tug_list,key=return_type)
        other = sorted(other._tug_list,key=return_type)
        if len(self) != len(other):
            return (False)
        else:
            for i in range(len(self)):
                if self[i]._type != other[i]._type:
                    return (False)
        return (True)
    
    def ___le__(self, other):
        return (not(self > other))

    def __ne__(self, other):
        return (not(self == other))
        
    def __gt__(self, other):
        self = sorted(self._tug_list,key = return_type)
        other = sorted(other._tug_list,key = return_type)
        if len(self) != len(other) :
            return (False)
        else:
            for i in range(len(self)):
                if self[i]._type > other[i]._type:
                    return(True)
        return (False)   
           
    def __ge__(self, other):
        return ((self > other) or (self == other))

class SHIP():
    """Characteristics of ships would be considered in planning
    
    Attributes:
        _id (int):        ID of the ship
        _pos ((int,int)): current position of the ship, a pair of latitude and lontitude
        _weight (int):    ship weight
        _velo (int):      velocity, default 10kt
    """
    
    def __init__(self, ship_id, cur_pos, weight, velo):
        self._ship_id = ship_id
        self._pos     = cur_pos
        self._weight  = weight
        self._velo    = velo
    def update_start_time(self):
        max_arrival_time = 0
        for i in range(len(self._tug)):
                if (max_arrival_time < self._tug[i] + count_move_time(_tug[i]._pos, self._from)):
                    max_arrival_time = self._tug[i] + count_move_time(_tug[i]._pos, self._from
        return max_arrival_time
        
class TASK():
    """
    Attributes:
       _ship (SHIP):               ship 
       _task_state (TASK_STATE):   task state
       _ship_state (SHIP_STATE):   ship state
       _start_time (datetime):     starting time
       _work_time (datetime) :     working time predicted by regression model 
       _from (int):                starting place, pier number or port
       _to (int):                  final place, pier number or port
       _side (SIDE):               a ship parks with its left or right side
    """
    _work_time = None
    _tug = []
    _task_state = TASK_STATE.UNPROCESSED_UNASSIGNED
    
    def __init__(self, ship, task_state, ship_state, start_time, start, dest, side=SIDE.OPTIONAL):
        self._ship       = ship
        self._ship_state = ship_state
        self._start_time = start_time
        self._from       = start
        self._to         = dest
        self._side       = side

        
def find_required_tug(weight):
    """
    Args:
        weight(TASK._ship._weight) : ship weight
    Returns:
        required_tug_list([CHARGE_TYPE]): a list of required tug list
    """
    if weight < 5000:
        return([[117],[117,117]])
    elif weight > 5000 and weight <= 10000:
         return ([[118],[117,117]])
    elif weight > 10000 and weight <= 15000:
        return ([[118],[118,117]])
    elif weight > 15000 and weight <= 30000:
        return ([[119],[118,119]])
    elif weight > 30000 and weight <= 45000:
        return ([[120],[120,120]])
    elif weight > 45000 and weight <= 60000:
        return ([[120],[120,120]])
    elif weight > 60000 and weight <= 100000:
        return ([[130],[130,130]])
    elif weight > 100000:
        return ([130,130])
    
        
    
def find_possible_set(tugs,required_tugs_list):
    """
        Args:
            tugs([TUG]):a list of available tugs
            required_tug_list([CHARGE_TYPE]):a list of required_tug_list
        Returns:
     """
    all_num = []
    for i in range(len(required_tugs_list)):
        all_num.append(len(required_tugs_list[i]))
    all_num = list(set(all_num))
    
    all_possible = []
    for i in range(len(all_num)):
        n = all_num[i]
        all_possible.extend(list(it.combinations(tugs, n)))
    return(all_possible)

def charge_type_ge(tug_set,required_tug_list):
    a = False
    for i in range(len(required_tug_list)):
        if len(tug_set) == len(required_tug_list[i]):
            a = True
            for j in range(len(tug_set)):
                if tug_set[j]._type < required_tug_list[i][j]:
                    return False
    return (True and a)

def find_best_set(tug_set,Task): 
    max_profit = 0
    work_time = 0
    best_set = []
    alternative_set = []
    for k in range(len(tug_set)):
        #function: regression, max_move_time,count_profit
        #return: datetime
        work_time = regression(Task,tug_set[k])    
        move_time = max_arrival_time(Task, tug_set[k])
        total_profit = count_profit(Task , work_time , move_time, tug_set[k]) 
        if  max_profit <= total_profit:
            max_profit = total_profit
            best_set = tug_set[k]
    return ({ "work_time" : work_time, "best_set" : best_set})          

def max_arrival_time(Task, tug_set):
    max_arrival_time = datetime()
    for i in range(len(tug_set)):
        move_time = count_move_time(tug_set[i]._pos,Task._from)
        max_arrival_time = max(max_arrival_time, tug_set[i]._next_arrival_time + move_time)
    return (max_arrival_time)
    
def regression(Task,tug_set[k]):
    ## not be completed
    df = pd.DataFrame([Task._ship_state,])
    pass

def charge_type_to_price(tug_list, required_tug_list): ##會找出各艘拖船較低的價錢總和
    price = 0
    price_dict = {117: 7395, 118: 10846, 119: 19720, 120: 22310, 130: 32000}
    for i in range(len(tug_list)):
        price += min(price_dict[required_tug_list._type],price_dict[tug_list[i]._type])
        
    return (price)
    
    
    

## return type : Float
def count_profit(Task , work_time , tug_set):
    oil_price = 0 # ?????? 
    total_move_time = 0
    total_profit = 0
    revenue = 0
    
    charge = charge_type_to_price(tug_set,find_required_tug(Task._ship._weight)[len(tug_set)-1]) 
    
    for i in range(len(tug_set)):
        total_move_time += count_move_time(tug_set[i]._pos, Task._from, Task._to)
    
    if work_time <= 60:
        return(charge)
    else:
        revenue = math.ceil(work_time / 30) * (float(charge)/2)

    total_profit += (revenue - total_move_time * oil_price) 
    return (total_profit) 

def harbor_to_pos(pier):
    ## State is IN
    harbor_pos = { 9001 : (22.616677,120.265942), 
                   9002 : (22.552638,120.316716)}
    
    if state == SHIP_STATE.IN:
        return(harbor_pos[pier])
    else:
        return(harbor_pos[pier])
    
def pier_to_pos(pier,df):
    return(df[int(pier),"經緯度"])
        

## return type : timedelta
## velo scale : 節
def count_move_time(state, _from,_to, velo = 10):
    if state == SHIP_STATE.IN
        _to = harbor_to_pos(_to)
    else:
        _to = pier_to_pos(_to)
    dis = count_dis(_from[0], _from[1], _to[0], _to[1])      
    return ((dis) / 1852 * 60 / velo)


def count_dis(base_lng,base_lat,lng,lat):
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371
    return c * r * 1000

    
#測試資料   ### 要加上next_available_time ㄡㄡㄡ
a = TUG(106,1001,117,1800)
b = TUG(108,1002,117,1800)
c = TUG(109,1030,118,2400)
d = TUG(112,1056,118,2400)
e = TUG(145,1023,118,2400)
f = TUG(165,8061,119,3200)
g = TUG(166,9032,120,4400)

A = SHIP(10053,1004,15300,10)
B = SHIP(40034,8062,25300,10)
C = SHIP(67803,1032,80000,12)
D = SHIP(46784,8061,34567,10)
E = SHIP(66734,1004,67500,10)
F = SHIP(55434,1006,36000,10)
G = SHIP(66666,1020,10890,12)
H = SHIP(43253,1035,34589,13)

T1 = TASK(A, 0, 0, datetime.now() + timedelta(minutes = 20) , 8062 ,1004, side=SIDE.OPTIONAL)
T2 = TASK(B, 0, 0, datetime.now() + timedelta(minutes = -20) , 8062 ,1004, 'L')
T3 = TASK(C, 0, 0, datetime.now() + timedelta(minutes = 40) , 8062 ,1004, side=SIDE.OPTIONAL)
T4 = TASK(D, 0, 0, datetime.now() + timedelta(minutes = 15) , 8062 ,1004, 'R')
T5 = TASK(E, 0, 0, datetime.now() + timedelta(minutes = 10) , 8062 ,1004, 'L')
T6 = TASK(F, 0, 0, datetime.now() + timedelta(minutes = -19) , 8062 ,1004, 'L')
T7 = TASK(G, 0, 0, datetime.now() + timedelta(minutes = 19) , 8062 ,1001, 'R')
T8 = TASK(H, 0, 0, datetime.now() + timedelta(minutes = 39) , 8062 ,1001, side=SIDE.OPTIONAL)

Tugs = [a,b,c,d,e,f,g]
Ships = [A,B,C,D,E,F,G,G]
Task = [T1,T2,T3,T4,T5,T6,T7,T8]
pos_of_pier = pd.read_excel("pos_of_pier.xlsx",index_col = "代號")

def greedy_dispatch(tasks):
    """
    Args: 
        tasks ([Task]): a list which stores the tasks to be planned
            
    Returns:
        tasks ([Task]): a list which stores the current processing and unprocessed tasks
    """ 
    # 1. 挑出task在一小時之內的task
    tmp = []
    for i in range(len(tasks)):
        if(task[i]._task_state == TASK_STATE.UNPROCESSED_UNASSIGNED \
        || task[i]._task_state = TASK_STATE.UNPROCESSED_ASSIGNED): 
            if((task[i]._start_time >= datetime.now()) && \
            task[i]._start_time - datetime.now() <= timedelta(hours=1)):
                tmp.append(tasks[i])
    tasks = tmp        
        

    # 2. 挑出free的tug or next avliable time 在一小時之前的task
    tugs = []
    for i in Tugs:
        if Tugs[i]._state.value ==  TUGSTATE.FREE:
            tugs.append(Tugs[i])
        elif Tugs[i]._state.value ==  TUGSTATE.BUSY:
            if Tugs[i]._next_available_time >=  datetime.datetime.now() && \
               Tugs[i]._next_available_time - datetime.now() <= timedelta(hours = 1):
                   tugs.append(Tugs[i])  
            
    
    #3.order all the task by start_time
    tasks = sorted(tasks,key = Task._start_time)
    #利用噸位尋找收費型號
    for i in Tasks:
        #required_tug_list (type):[收費型號] ex: [117,118]
        required_tugs_list = find_required_tug(Tasks[i]._weight)
        # possible set 是所有可用到的拖船組合
        # type: [TUG]
        # function: find_possible_set : 把已經預排的與正在工作的拖船剔除在拖船組合之外
        # 考慮_next_available_time  
        possible_set = find_possible_set(tugs,required_tugs_list）
        max_profit = 0
        alternative_set = []
        
        for j in range(len(possible_set)):
            ##  Set comparision
            if not(charge_type_ge(possible_set[j],required_tug_list)):
            # alternative_set 是指小於符合拖船派遣規則的拖船組合
                alternative_set.append(possible_set[j])

        possible_set = list(set(possible_set) - set(alternative_set)) 
        
        if len(possible_set) == 0: 
            result = find_best_set(alternative_set,Tasks[i])
             best_set = result['best_set']
        else:
            result = find_best_set(possible_set,Tasks[i])
            best_set = result['best_set']
             
        if len(best_set) == 0:
            continue
            
        ### 更改task 狀態 + tug._next_available_time 
        Tasks[i]._task_state = TASK.UNPROCESSED_ASSIGNED
        Task[i]._work_time = result['work_time']
        Task[i]._tug = best_set
        Task[i].update_start_time()
        
        for j in range(len(best_set)):
            for k in range(len(tugs)):
                ## If the tug is assigned update the next_available time
                if tugs[k]._id == best_set[j]:
                    tugs[k]._next_available_time =  + result['work_time']


In [18]:
# 拖船分派演算法

## 貪婪 (greedy algorithm)
import itertools as it 
import pandas as pd
import math

from datetime import datetime  
from datetime import timedelta

from enum import Enum, IntEnum

## Self-defined function
# from WorkTimePrediction import *

class Event():
    def __init__():
        pass
    def event_handler():
        pass

class Event(Enum):
    """All kinds of events which trigger planning algorithm
    
    Attributes:
        ROUTINE:            dispatch every one hour
        SHIP_DELAY:         delay caused by the ship
        TUG_DELAY:          delay caused by the tug
        UNIDENTIFIED_DELAY: delay occurs under unidentified situation
        MISMATCH:           the previous assignment is refused by the pilot
        BEFOREHAND:         any of the tugs finish task in advance
        CANCELED:           the task is canceled
        ADD_TUG:            need additional tugs on the half way
    """
    
    ROUTINE            = 'routine' 
    CANCELED           = 'canceled'
    TUG_DELAY          = 'tug_delay'
    SHIP_DELAY         = 'ship_delay'
    UNIDENTIFIED_DELAY = 'unidentified_delay'
    
    MISMATCH           = 'mismatch'
    BEFOREHAND         = 'beforehand'
    ADD_TUG            = 'add_tug'
    
    
class TUG_STATE(Enum):
    """States of tugs which are considered when planning
    
    Attributes:
        FREE:        a tug which is ready to work immediately
        BUSY:        a tug which is busy working on a specific tesk
        UNAVAILABLE: a tug which is either broken or out of order
    """
    FREE        = 0
    BUSY        = 1
    UNAVAILABLE = 2

class CHARGE_TYPE(IntEnum):
    """Types of tugs which are used in charging and planning.
        Values of attributes correpond to weights according to the dispatching rule
    """
    
    TYPE_117 = 117
    TYPE_118 = 118
    TYPE_119 = 119
    TYPE_120 = 120
    TYPE_0   = 130
    
class TASK_STATE(IntEnum):
    """States of task describing 
        if tugs are assigned (UNASSIGNED or ASSIGNED)
        if the task is finished (UNPROCESSED, PROCESSING, or PROCESSED)
    """
    
    UNPROCESSED_UNASSIGNED = 0
    UNPROCESSED_ASSIGNED   = 1
    PROCESSING_ASSIGNED    = 2
    PROCESSED              = 3
    
class SIDE(Enum):
    LEFT     = 'L' 
    RIGHT    = 'R'
    OPTIONAL = 'O'

class SHIP_STATE(Enum):
    IN        = 'I'
    OUT       = 'O'
    TRANSFER  = 'T'
    
class TUG():
    """Main entities to be planned
    
    Attributes:
        _id (int):           ID of the tug
        _pos ((float, float)):   current position of the tug, a pair of latitude and lontitude
        _type (CHARGE_TYPE): charging type
        _hp (int):           horsepower of the tug
        _velo (int):         velocity, default 10kt
        _state (TUG_STATE):  current state of the tug
        _next_available_time: if tug state is busy,we will recorded the time which it will finish the task
    """
    
    def __init__(self, tug_id, cur_pos, charge_type, 
                hp, next_available_time ,state=TUG_STATE.FREE, velo=10):
        self._tug_id = tug_id
        self._pos    = cur_pos
        self._type   = charge_type
        self._hp     = hp
        self._state  = state
        self._velo   = velo
        self._next_avaliable_time = next_available_time
        
def return_type(elem):
            return (elem._type)
class TUG_LIST(object):
    
    _tug_list = []
    
    def __init__(self, tug_list):
        self._tug_list = tug_list
        
    def __lt__(self, other):
        return(not(self > other) and not(self == other))

    def __eq__(self, other):
        self = sorted(self._tug_list,key=return_type)
        other = sorted(other._tug_list,key=return_type)
        if len(self) != len(other):
            return (False)
        else:
            for i in range(len(self)):
                if self[i]._type != other[i]._type:
                    return (False)
        return (True)
    
    def ___le__(self, other):
        return (not(self > other))

    def __ne__(self, other):
        return (not(self == other))
        
    def __gt__(self, other):
        self = sorted(self._tug_list,key = return_type)
        other = sorted(other._tug_list,key = return_type)
        if len(self) != len(other) :
            return (False)
        else:
            for i in range(len(self)):
                if self[i]._type > other[i]._type:
                    return(True)
        return (False)   
           
    def __ge__(self, other):
        return ((self > other) or (self == other))

class SHIP():
    """Characteristics of ships would be considered in planning
    
    Attributes:
        _id (int):        ID of the ship
        _pos ((int,int)): current position of the ship, a pair of latitude and lontitude
        _weight (int):    ship weight
        _velo (int):      velocity, default 10kt
    """
    
    def __init__(self, ship_id, cur_pos, weight, velo):
        self._ship_id = ship_id
        self._pos     = cur_pos
        self._weight  = weight
        self._velo    = velo
        
class TASK():
    """
    Attributes:
       _ship (SHIP):               ship 
       _task_state (TASK_STATE):   task state
       _ship_state (SHIP_STATE):   ship state
       _start_time (datetime):     starting time
       _work_time (datetime) :     working time predicted by regression model 
       _from (int):                starting place, pier number or port
       _to (int):                  final place, pier number or port
       _side (SIDE):               a ship parks with its left or right side
    """
    _work_time = None
    _tug = []
    _task_state = TASK_STATE.UNPROCESSED_UNASSIGNED
    
    def __init__(self, ship, task_state, ship_state, start_time, start, dest, side=SIDE.OPTIONAL):
        self._ship       = ship
        self._ship_state = ship_state
        self._start_time = start_time
        self._from       = start
        self._to         = dest
        self._side       = side
    def update_start_time(self):
        max_arrival_time = 0
        for i in range(len(self._tug)):
                if (max_arrival_time < self._tug[i] + count_move_time(self._tug[i]._pos, self._from)):
                    max_arrival_time = self._tug[i] + count_move_time(self._tug[i]._pos, self._from)
        self._start_time = max_arrival_time
        
     

In [50]:
#測試資料   ### 要加上next_available_time ㄡㄡㄡ
a = TUG(106,1001,117,1800,5)
b = TUG(108,1002,117,1800,30)
c = TUG(109,1030,118,2400,1)
d = TUG(112,1056,118,2400,20)
e = TUG(145,1023,118,2400,0)
f = TUG(165,8061,119,3200,0)
g = TUG(166,9032,120,4400,10)

A = SHIP(10053,1004,15300,10)
B = SHIP(40034,8062,25300,10)
C = SHIP(67803,1032,80000,12)
D = SHIP(46784,8061,34567,10)
E = SHIP(66734,1004,67500,10)
F = SHIP(55434,1006,36000,10)
G = SHIP(66666,1020,10890,12)
H = SHIP(43253,1035,34589,13)

T1 = TASK(A, 0, 0, datetime.now() + timedelta(minutes = 20) , 8062 ,1004, side=SIDE.OPTIONAL)
T2 = TASK(B, 0, 0, datetime.now() + timedelta(minutes = -20) , 8062 ,1004, 'L')
T3 = TASK(C, 0, 0, datetime.now() + timedelta(minutes = 40) , 8062 ,1004, side=SIDE.OPTIONAL)
T4 = TASK(D, 0, 0, datetime.now() + timedelta(minutes = 15) , 8062 ,1004, 'R')
T5 = TASK(E, 0, 0, datetime.now() + timedelta(minutes = 10) , 8062 ,1004, 'L')
T6 = TASK(F, 0, 0, datetime.now() + timedelta(minutes = -19) , 8062 ,1004, 'L')
T7 = TASK(G, 0, 0, datetime.now() + timedelta(minutes = 19) , 8062 ,1001, 'R')
T8 = TASK(H, 0, 0, datetime.now() + timedelta(minutes = 39) , 8062 ,1001, side=SIDE.OPTIONAL)


In [58]:
def find_required_tug(weight):
    """
    Args:
        weight(TASK._ship._weight) : ship weight
    Returns:
        required_tug_list([CHARGE_TYPE]): a list of required tug list
    """
    if weight <= 5000:
        return([[117],[117,117]])
    elif weight > 5000 and weight <= 10000:
         return ([[118],[117,117]])
    elif weight > 10000 and weight <= 15000:
        return ([[118],[118,117]])
    elif weight > 15000 and weight <= 30000:
        return ([[119],[118,119]])
    elif weight > 30000 and weight <= 45000:
        return ([[120],[120,120]])
    elif weight > 45000 and weight <= 60000:
        return ([[120],[120,120]])
    elif weight > 60000 and weight <= 100000:
        return ([[130],[130,130]])
    elif weight > 100000:
        return ([[130,130]])

In [78]:

def charge_type_to_price(tug_list, required_tug_list): ##會找出各艘拖船較低的價錢總和
    price = 0
    price_dict = {117: 7395, 118: 10846, 119: 19720, 120: 22310, 130: 32000}
    for i in range(len(tug_list)):
        price += min(price_dict[required_tug_list[i]],price_dict[tug_list[i]._type])
        
    return (price)

## return type : Float
def count_profit(Task , work_time , tug_set):
    oil_price = 0 # ?????? 
    total_move_time = 0
    total_profit = 0
    revenue = 0
    required_tug_set_all = find_required_tug(Task._ship._weight)
    required_tug_set = None
    
    for i in range(len(required_tug_set_all)):
        if len(required_tug_set_all[i]) == len(tug_set):
            required_tug_set = required_tug_set_all[i]
            break
            
    
    if required_tug_set == None:
        charge = charge_type_to_price(tug_set,tug_set) 
    else:
        charge = charge_type_to_price(tug_set,required_tug_set) 
    
    
    for i in range(len(tug_set)):
        total_move_time += count_move_time(tug_set[i]._pos, Task._from, Task._to)
    
    if work_time <= 60:
        return(charge)
    else:
        revenue = math.ceil(work_time / 30) * (float(charge)/2)

    total_profit += (revenue - total_move_time * oil_price) 
    return (total_profit)


# charge_type_to_price([a,c,d],[117,117,118])
# tug_set = [a,d,f]
# len(tug_set)
# print(find_required_tug(T1._ship._weight))
count_profit(T1, 30, [a,b])


[118, 119]


NameError: name 'count_move_time' is not defined